## Extracting Rules from a Language Model 

In [1]:
import numpy as np
import random
import timeit
from rule_extractor_original.Lenses_dataset.Horn import *
from binarize_features import *
from transformers import pipeline
from helper_functions import *
from scipy.special import comb
import pickle

122


c:\Users\sophi\Miniconda3\envs\MT\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Import variables and sizes with the binarizer

In [4]:
country_file = 'data/country_list_continents_new.csv'
occ_file = 'data_new/occupations_subset.csv'
#occ_file = 'data/occupations_subset.csv'
binarizer = Binarizer(country_file, 5, occ_file)
print(binarizer.occupation_lookup)

{'nurse': 0, 'fashion_designer': 1, 'dancer': 2, 'footballer': 3, 'industrialist': 4, 'boxer': 5, 'singer': 6, 'violinist': 7}


In [5]:
print(pd.read_csv('data_new/occupations_subset.csv').to_numpy().flatten())

['nurse' 'fashion_designer' 'dancer' 'footballer' 'industrialist' 'boxer'
 'singer' 'violinist']


In [6]:
models = ['roberta-base', 'roberta-large', 'bert-base-cased', 'bert-large-cased']
# add 2 dimensions for the gender variables (last variables in the vector)
dim = sum(binarizer.lengths.values()) + 2
V = define_variables(dim)
language_model = models[0]
#seed = 123 #reproducability
epsilon = 0.3
delta = 0.1

Sample size code taken from Lenses.ipynb

In [7]:
def get_eq_sample_size(n_variables=dim, epsilon=epsilon, delta=delta):
    return int((4/epsilon) * (log( log( Pow(2,comb(n_variables,n_variables/2)) ) )/delta) + 1)
print(get_eq_sample_size())
print(dim)

1926
24


### Create classified samples from the Language Model

In [6]:
def get_random_sample(length, allow_zero = True, amount_of_true=1):
    vec = np.zeros(length, dtype=np.int8)
    # allow for all zeroes: one extra sample length and if its out of index range, use all zeroes vector (equal possibility)
    if allow_zero:
        idx = random.sample(range(length + 1), k=amount_of_true)
    else:
        idx = random.sample(range(length), k=amount_of_true)
    for i in idx:
        if i < length:
            vec[i] = 1
    return list(vec)

In [7]:
def get_label(classification, gender):
    if (gender[0] == 1 and classification == 0) or (gender[1] == 1 and classification == 1):
        return 1
    else:
        return 0

In [8]:
#len(V) = num_variables
attributes = ['birth', 'continent', 'occupation']
#new tactic: 
#num_variables inherited by binarizer that has to be initialized
def create_classified_sample_from_LM(lm : str, sample_size : int, binarizer : Binarizer, unmasker, verbose = False):
    dataset = []
    # create all the samples randomly by specific sampling strategy and classify them immediatley 
    # -> there is no time savings in using a batch of samples at once and therefore predictions can be done immediately
    for i in range(sample_size):
        vec = []
        for att in attributes:
            # get the appropriate vector for each attribute and tie them together in the end
            vec = [*vec, *get_random_sample(binarizer.lengths[att], allow_zero=True)]
        s = binarizer.sentence_from_binary(vec)
        if verbose:
            print(s)
        # classification: 0 = female, 1 = male
        classification = get_prediction(lm_inference(unmasker, s, model=lm), binary = True)
        # get random gender as a fourth attribute (in the end of the vector) as a one-hot-encoding with two dimensions: [female, male]
        gender_vec = get_random_sample(2, allow_zero=False)
        vec = [*vec, *gender_vec]
        # if the sampled gender is equal the classification (correctly classified) then we return 1 as 'is valid sentence' 
        # if sampled gender and classification don't match, the sample is not valid and we return 0 as a label
        label = get_label(classification, gender_vec)
        if verbose:
            print((vec,classification, gender_vec, label))
        dataset.append((vec,label))
    return dataset
dataset = create_classified_sample_from_LM(language_model, 5, binarizer, pipeline('fill-mask', model=language_model), verbose = True)

<mask> was born after 1976 in North America and is a not known occupation.
([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 1, [0, 1], 1)
<mask> was born after 1976 in South America and is a footballer.
([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0], 1, [1, 0], 0)
<mask> was born between 1892 and 1934 in South America and is a banker.
([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0], 1, [1, 0], 0)
<mask> was born after 1976 in Africa and is a dancer.
([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0], 0, [1, 0], 1)
<mask> was born between 1934 and 1956 in South America and is a mathematician.
([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], 1, [1, 0], 0)


### Define equivalence and membership queries for language model

Adjust methods for equivalence and membership queries so that they fit with inputs and outputs for a language model. 
1. use modified `create_classified_sample_from_LM`
2. replace prediction in MQ with prediction from language model

In [9]:
def custom_EQ(H, lm, unmasker, V, bad_nc, binarizer : Binarizer):
    sample = create_classified_sample_from_LM(lm, get_eq_sample_size(), binarizer, unmasker)
    h = true
    if len(H):
        h = set2theory(H)
    for (a,l) in sample:
        if l == 0 and evaluate(h,a,V) and a not in bad_nc:
            return a
        if l == 1 and not evaluate(h,a,V):
            sample.remove((a,l))
            return a
    return True

def custom_MQ(assignment, lm, unmasker, binarizer : Binarizer):
    vec = assignment[:-2]
    gender_vec = assignment[-2:]
    s = binarizer.sentence_from_binary(vec)
    classification = get_prediction(lm_inference(unmasker, s, model=lm), binary = True)
    label = get_label(classification, gender_vec)
    res  = ( True if label == 1
                else False)
    return res

### Modify `extract_horn_with_queries` 

In [10]:
def extract_horn_with_queries_1(lm, V, iterations, binarizer, background, verbose = 0):
    bad_pc = []
    bad_ne =[]
    unmasker = pipeline('fill-mask', model=lm)
    mq = lambda a : custom_MQ(a, lm, unmasker, binarizer)
    eq = lambda a : custom_EQ(a, lm, unmasker, V, bad_ne, binarizer)

    start = timeit.default_timer()
    h = learn(V, mq, eq, bad_ne, bad_pc, background = background, iterations=iterations, verbose = verbose)
    stop = timeit.default_timer()
    runtime = stop-start

    runtime_per_iteration = runtime / iterations
    return (h,runtime,runtime_per_iteration)


## Run experiments 

Make a background that clarifies that some features can't appear at the same time to allow for the one-hot-encoding.

In [8]:
print(V)
print(binarizer.lengths)

[v0, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, v17, v18, v19, v20, v21, v22, v23]
{'birth': 5, 'continent': 9, 'occupation': 8}


In [10]:
background_string = "{"
for idx, length in enumerate(binarizer.lengths.values()):
    for i in range(length):
        for j in range(i+1,length):
            add = 0
            if idx == 1:
                add = 5
            elif idx == 2:
                add = 5 + 9
            background_string = background_string + "(~(V[" + str(i+add) + "] & V[" + str(j+add) + "])),\n"
background_string = background_string + "(~(V[22] & V[23])),\n"
background_string = background_string + "}"
print(background_string)

{(~(V[0] & V[1])),
(~(V[0] & V[2])),
(~(V[0] & V[3])),
(~(V[0] & V[4])),
(~(V[1] & V[2])),
(~(V[1] & V[3])),
(~(V[1] & V[4])),
(~(V[2] & V[3])),
(~(V[2] & V[4])),
(~(V[3] & V[4])),
(~(V[5] & V[6])),
(~(V[5] & V[7])),
(~(V[5] & V[8])),
(~(V[5] & V[9])),
(~(V[5] & V[10])),
(~(V[5] & V[11])),
(~(V[5] & V[12])),
(~(V[5] & V[13])),
(~(V[6] & V[7])),
(~(V[6] & V[8])),
(~(V[6] & V[9])),
(~(V[6] & V[10])),
(~(V[6] & V[11])),
(~(V[6] & V[12])),
(~(V[6] & V[13])),
(~(V[7] & V[8])),
(~(V[7] & V[9])),
(~(V[7] & V[10])),
(~(V[7] & V[11])),
(~(V[7] & V[12])),
(~(V[7] & V[13])),
(~(V[8] & V[9])),
(~(V[8] & V[10])),
(~(V[8] & V[11])),
(~(V[8] & V[12])),
(~(V[8] & V[13])),
(~(V[9] & V[10])),
(~(V[9] & V[11])),
(~(V[9] & V[12])),
(~(V[9] & V[13])),
(~(V[10] & V[11])),
(~(V[10] & V[12])),
(~(V[10] & V[13])),
(~(V[11] & V[12])),
(~(V[11] & V[13])),
(~(V[12] & V[13])),
(~(V[14] & V[15])),
(~(V[14] & V[16])),
(~(V[14] & V[17])),
(~(V[14] & V[18])),
(~(V[14] & V[19])),
(~(V[14] & V[20])),
(~(V[14] & V[21])),

In [13]:
background = {(~(V[0] & V[1])),
(~(V[0] & V[2])),
(~(V[0] & V[3])),
(~(V[0] & V[4])),
(~(V[1] & V[2])),
(~(V[1] & V[3])),
(~(V[1] & V[4])),
(~(V[2] & V[3])),
(~(V[2] & V[4])),
(~(V[3] & V[4])),
(~(V[5] & V[6])),
(~(V[5] & V[7])),
(~(V[5] & V[8])),
(~(V[5] & V[9])),
(~(V[5] & V[10])),
(~(V[5] & V[11])),
(~(V[5] & V[12])),
(~(V[5] & V[13])),
(~(V[6] & V[7])),
(~(V[6] & V[8])),
(~(V[6] & V[9])),
(~(V[6] & V[10])),
(~(V[6] & V[11])),
(~(V[6] & V[12])),
(~(V[6] & V[13])),
(~(V[7] & V[8])),
(~(V[7] & V[9])),
(~(V[7] & V[10])),
(~(V[7] & V[11])),
(~(V[7] & V[12])),
(~(V[7] & V[13])),
(~(V[8] & V[9])),
(~(V[8] & V[10])),
(~(V[8] & V[11])),
(~(V[8] & V[12])),
(~(V[8] & V[13])),
(~(V[9] & V[10])),
(~(V[9] & V[11])),
(~(V[9] & V[12])),
(~(V[9] & V[13])),
(~(V[10] & V[11])),
(~(V[10] & V[12])),
(~(V[10] & V[13])),
(~(V[11] & V[12])),
(~(V[11] & V[13])),
(~(V[12] & V[13])),
(~(V[14] & V[15])),
(~(V[14] & V[16])),
(~(V[14] & V[17])),
(~(V[14] & V[18])),
(~(V[14] & V[19])),
(~(V[14] & V[20])),
(~(V[14] & V[21])),
(~(V[14] & V[22])),
(~(V[14] & V[23])),
(~(V[15] & V[16])),
(~(V[15] & V[17])),
(~(V[15] & V[18])),
(~(V[15] & V[19])),
(~(V[15] & V[20])),
(~(V[15] & V[21])),
(~(V[15] & V[22])),
(~(V[15] & V[23])),
(~(V[16] & V[17])),
(~(V[16] & V[18])),
(~(V[16] & V[19])),
(~(V[16] & V[20])),
(~(V[16] & V[21])),
(~(V[16] & V[22])),
(~(V[16] & V[23])),
(~(V[17] & V[18])),
(~(V[17] & V[19])),
(~(V[17] & V[20])),
(~(V[17] & V[21])),
(~(V[17] & V[22])),
(~(V[17] & V[23])),
(~(V[18] & V[19])),
(~(V[18] & V[20])),
(~(V[18] & V[21])),
(~(V[18] & V[22])),
(~(V[18] & V[23])),
(~(V[19] & V[20])),
(~(V[19] & V[21])),
(~(V[19] & V[22])),
(~(V[19] & V[23])),
(~(V[20] & V[21])),
(~(V[20] & V[22])),
(~(V[20] & V[23])),
(~(V[21] & V[22])),
(~(V[21] & V[23])),
(~(V[22] & V[23])),
(~(V[24] & V[25])),
}

In [14]:
with open('data/background.txt', 'wb') as f:
    pickle.dump(background, f)

In [15]:
(h,runtime,runtime_per_iteration) = extract_horn_with_queries_1(language_model, V, 10, binarizer, background, verbose=2)
print(runtime,runtime_per_iteration)
with open(language_model + '_rules.txt', 'wb') as f:
    pickle.dump(h, f)

Iteration 10

hypothesis is {~(v6 & v8), ~(v20 & v23), ~(v10 & v11), ~(v18 & v23), ~(v12 & v5), ~(v5 & v7), ~(v15 & v22), ~(v14 & v16), ~(v11 & v6), ~(v19 & v22), ~(v15 & v20), ~(v13 & v9), ~(v0 & v1), ~(v19 & v20), ~(v10 & v9), ~(v14 & v17), ~(v16 & v19), ~(v11 & v7), ~(v24 & v25), ~(v15 & v23), ~(v10 & v12), ~(v6 & v9), ~(v0 & v2), ~(v8 & v9), ~(v2 & v3), ~(v13 & v8), ~(v10 & v8), ~(v14 & v15), ~(v16 & v17), ~(v20 & v21), ~(v1 & v4), ~(v15 & v16), ~(v14 & v18), ~(v0 & v4), ~(v0 & v3), ~(v13 & v7), ~(v15 & v17), ~(v10 & v13), ~(v12 & v9), ~(v13 & v6), ~(v14 & v23), ~(v16 & v23), ~(v16 & v18), ~(v5 & v8), ~(v3 & v4), ~(v21 & v22), ~(v7 & v9), ~(v19 & v23), ~(v11 & v8), ~(v20 & v22), ~(v15 & v18), ~(v17 & v23), ~(v14 & v21), ~(v13 & v5), ~(v10 & v5), ~(v1 & v2), ~(v6 & v7), ~(v21 & v23), ~(v22 & v23), ~(v18 & v21), ~(v14 & v19), ~(v5 & v6), ~(v7 & v8), ~(v17 & v21), ~(v16 & v21), ~(v12 & v8), ~(v11 & v9), ~(v18 & v19), ~(v12 & v6), ~(v2 & v4), ~(v17 & v19), ~(v11 & v12), ~(v10 & v6), ~(